In [1]:
from datasets import load_dataset, Dataset, Features, Array2D
import glob
import os
import torch
import torchaudio

In [2]:
from datasets import load_dataset, Dataset
import glob
import os
import torch
import torchaudio

In [3]:
sources = glob.glob("MYRiAD_V2/audio/SRC/*.wav")
cocktail_party = [f"CP{i}.wav" for i in range(6)];
source_list = [suffix[-6:] for suffix in sources] + cocktail_party 
inner = [f"CMA10_{a}_" for a in [0,90,180,-90]]
outer = [f"CMA20_{a}_" for a in [0,45,90,135,180,-135,-90,-45]]
channel_list = inner + outer
ref_channel = channel_list[0]

In [4]:
audio = []

for source in source_list:
    query = ref_channel+source
    prefix_list = glob.glob("MYRiAD_V2/audio/*/*/*/"+query)
    prefix_list = [match[:-len(query)] for match in prefix_list]
    for prefix in prefix_list:
        x = [];
        try:
            for channel in channel_list:
                filename = prefix + channel + source
                x.append(torchaudio.load(filename)[0][0])
            x = torch.stack(x)
            audio.append(x)
        except:
            pass

In [5]:
dict = {f"channel_{i}": [recording[i] for recording in audio] for i in range(12)}
dataset = Dataset.from_dict(dict)
dataset = dataset.with_format("torch")

In [ ]:
dataset.push_to_hub("danjacobellis/CMA12")

Pushing dataset shards to the dataset hub:   0%|          | 0/34 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]